# Hadoop Streaming assignment 2: Stop Words

The purpose of this task is to improve the previous "Word rating" program. You have to calculate how many stop words are there in the input dataset. Stop words list is in `/datasets/stop_words_en.txt` file. 

Use Hadoop counters to compute the number of stop words and total words in the dataset. The result is the percentage of stop words in the entire dataset (without percent symbol).

There are several points for this task:

1) As an output, you have to get the percentage of stop words in the entire dataset without percent symbol (correct answer on sample dataset is `41.603`).

2) As you can see in the Hadoop Streaming userguide "you will need to use `-files` option to tell the framework to pack your executable files as a part of a job submission."

3) Do not forget to redirect junk output to `/dev/null`.

4) You may modify mappers/reducers from "Word rating" task and parse its output to get the answer on "Stop Words" task.

5) You may use mapper/reducer to get `"Stop Words"` and `"Total Words"` amounts and redirect them to sys.stderr. After that you may redirect the output of MapReduce to the parsed function. In this function you may find rows correspond to these amounts and compute the percentage.

Here you can find the draft for the main steps of the task. You can use other methods to get the solution.

## Step 1. Create the mapper.

<b>Hint:</b> Create the mapper, which calculates Total word and Stop word amounts. You may redirect this information to sys.stderr. This will make it possible to parse these data on the next steps.

Example of the redirections:

`print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % count`

Remember about the Distributed cache. If we add option `-files mapper.py,reducer.py,/datasets/stop_words_en.txt`, then `mapper.py, reducer.py` and `stop_words_en.txt` file will be in the same directory on the datanodes. Hence, it is necessary to use a relative path `stop_words_en.txt` from the mapper to access this txt file.

In [3]:
!hdfs dfs -ls /datasets/stop_words_en.txt .

-rw-r--r--   1 jovyan supergroup       1914 2018-08-01 17:15 /datasets/stop_words_en.txt
Found 31 items
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 16:53 stopword_result_1546793348791328
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 17:05 stopword_result_1546794183590560
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 18:28 stopword_result_1546799165944677
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 18:34 stopword_result_1546799497038878
drwxr-xr-x   - jovyan supergroup          0 2019-01-04 17:42 test
drwxr-xr-x   - jovyan supergroup          0 2019-01-05 16:15 test_result
drwxr-xr-x   - jovyan supergroup          0 2019-01-02 12:50 wordcount_result_1546433333935161
drwxr-xr-x   - jovyan supergroup          0 2019-01-02 13:16 wordcount_result_1546434974465272
drwxr-xr-x   - jovyan supergroup          0 2019-01-02 13:21 wordcount_result_1546435279584368
drwxr-xr-x   - jovyan supergroup          0 2019-01-02 13:25 wordcount_result_1546435501138649
d

In [16]:
%%writefile mapper.py


import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

# Your code for reading stop words here
stopWords = open(path).read().split()

wordSum, stopWordSum = 0, 0

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)

    # Your code for mapper here.
    for word in words:
        if word in stopWords:
            stopWordSum += 1
            continue
            #print >> sys.stderr, "reporter:counter:Stop words,1"
        #wordSum += 1
        #print >> sys.stderr, "reporter:counter:Total words, 1"
    wordSum += len(words)
    
print >> sys.stderr, "reporter:counter:Total words,%d" % wordSum
print >> sys.stderr, "reporter:counter:Stop words,%d" % stopWordSum

print "wordSum-stopWordSum\t%d\t%d" % (wordSum, stopWordSum)

Overwriting mapper.py


## Step 2. Create the reducer.

Create the reducer, which will accumulate the information after the mapper step. You may implement the combiner if you want. It can be useful from optimizing and speed up your computations (see the lectures from the Week 2 for more details).

In [17]:
%%writefile reducer.py

# Your code for reducer here.
import sys

totalWords, totalStopWords = 0, 0

for line in sys.stdin:
    try:
        _, wordSum, stopWordSum = line.strip().split('\t')
    except:
        continue
    totalWords += int(wordSum)
    totalStopWords += int(stopWordSum)
    
print "wordSum-stopWordSum\t%d\t%d" % (totalWords, totalStopWords)
    

Overwriting reducer.py


In [20]:
%%bash

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...

OUT_DIR="stopword_result_"$(date +"%s%6N")
NUM_REDUCERS=1
LOGS="stderr_logs.txt"

#hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Stop wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,hdfs:///datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS
    
hdfs dfs -cat ${OUT_DIR}/*


wordSum-stopWordSum	11937375	4588771


In [19]:
!hdfs dfs -


Found 35 items
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 16:53 stopword_result_1546793348791328
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 17:05 stopword_result_1546794183590560
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 18:28 stopword_result_1546799165944677
drwxr-xr-x   - jovyan supergroup          0 2019-01-06 18:34 stopword_result_1546799497038878
drwxr-xr-x   - jovyan supergroup          0 2019-01-08 14:44 stopword_result_1546958521159072
drwxr-xr-x   - jovyan supergroup          0 2019-01-08 15:51 stopword_result_1546962541892332
drwxr-xr-x   - jovyan supergroup          0 2019-01-08 16:00 stopword_result_1546963051786177
drwxr-xr-x   - jovyan supergroup          0 2019-01-08 18:07 stopword_result_1546970690716924
drwxr-xr-x   - jovyan supergroup          0 2019-01-04 17:42 test
drwxr-xr-x   - jovyan supergroup          0 2019-01-05 16:15 test_result
drwxr-xr-x   - jovyan supergroup          0 2019-01-02 12:50 wordcount_result_1546433333935161

## Step 3. Create the parsed function.

<b>Hint:</b> Create the function, which will parse MapReduce sys.stderr for Total word and Stop word amounts.

The `./counter_process.py` script should do the following:

- parse hadoop logs from Stderr,

- retrieve values of 2 user-defined counters,

- compute percentage and output it into the stdout.

In [0]:
%%writefile counter_process.py

#! /usr/bin/env python

import sys

# Your functions may be here.



if __name__ == '__main__':
    # Your code here.

## Step 4. Bash commands

<b> Hints: </b> 

1) If you want to redirect standard output to txt file you may use the following argument in yarn jar:

```
yarn ... \
  ... \
  -output ${OUT_DIR} > /dev/null 2> $LOGS
```

2) For printing the percentage of stop words in the entire dataset you may parse the MapReduce output. Parsed script may be written in Python code. 

To get the result you may use the UNIX pipe operator `|`. The output of the first command acts as an input to the second command (see lecture file-content-exploration-2 for more details).

With this operator you may use command `cat` to redirect the output of MapReduce to ./counter_process.py with arguments, which correspond to the `"Stop words"` and `"Total words"` counters. Example is the following:

`cat $LOGS | python ./counter_process.py "Stop words" "Total words"`

Now something about Hadoop counters naming. 
 - Built-in Hadoop counters usually have UPPER_CASE names. To make the grading system possible to distinguish your custom counters and system ones please use the following pattern for their naming: `[Aa]aaa...` (all except the first letters should be in lowercase);
 - Another points is how Hadoop sorts the counters. It sorts them lexicographically. Grading system reads your first counter as Stop words counter and the second as Total words. Please name you counters in such way that Hadoop set the Stop words counter before the Total words. 
 
E.g. "Stop words" and "Total words" names are Ok because they correspond both requirements.

3) In Python code sys.argv is a list, which contains the command-line arguments passed to the script. The name of the script is in `sys.argv[0]`. Other arguments begin from `sys.argv[1]`.

Hence, if you have two arguments, which you send from the Bash to your python script, you may use arguments in your script with the following command:

`function(sys.argv[1], sys.argv[2])`

4) Do not forget about printing your MapReduce output in the last cell. You may use the next command:

`cat $LOGS >&2`

In [0]:
%%bash

OUT_DIR="coursera_mr_task2"$(date +"%s%6N")
NUM_REDUCERS=8
LOGS="stderr_logs.txt"

# Stub code for your job

# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
# ... \
#    -output ${OUT_DIR} > /dev/null 2> $LOGS
    
cat $LOGS | python ./counter_process.py "Stop words" "Total words"
cat $LOGS >&2

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
